### Source Part1: [Youtube#1](https://www.youtube.com/watch?v=mCkfYZdrWvs&ab_channel=BleedAI)
### Source Part1: [Youtube#2](https://www.youtube.com/watch?v=j7Tnmrc7hgE&ab_channel=BleedAI)
### Source Part3: [Youtube#3](https://www.youtube.com/watch?v=aKnaVwzR4nw&ab_channel=BleedAI)

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.interpolate import UnivariateSpline
import os

### Apply Mouse Click Events with OpenCV
* This is setting up the foundation to interact with the filter selection and parameter tuning with user interface
* So not much image processing will be introduced

#### `cv2.MouseEventTypes`: All of the supported mouse event by OpenCV
* Pair those event activation methods with the corresponding callback functions using `cv2.setMouseCallback()`
* Before that, we have to create a 'window' to pass into the callback function as a target, use `cv2.namedWindow(windowName: str)` 

In [2]:
# Create a named resizable window.
# This will create and open up a OpenCV image window.
# Minimize the window and run the next cells
# Do not close this window.
cv2.namedWindow("window1", cv2.WINDOW_NORMAL)

In [3]:
# Create a callback function that will draw a shape at the spot press by the mouse curser 
def drawShapes(event, x, y, flags, userdata):
    '''
    This function will draw circle and rectangle on a canvas and clear it based 
    on different mouse events.
    Args:
        event:    The mouse event that is captured.
        x:        The x-coordinate of the mouse pointer position on the window.
        y:        The y-coordinate of the mouse pointer position on the window.
        flags:    It is one of the MouseEventFlags constants.
        userdata: The parameter passed from the `cv2.setMouseCallback()` function.
    '''
    
    # Access the canvas from outside of the current scope.
    global canvas
    
    # Check if the left mouse button is pressed.
    if event == cv2.EVENT_LBUTTONDOWN:
        
        # Draw a circle on the current location of the mouse pointer.
        cv2.circle(img=canvas, center=(x, y), radius=50,
                   color=(113,182,255), thickness=-1)
        
    # Check if the right mouse button is pressed.
    elif event == cv2.EVENT_RBUTTONDOWN:
        
        # Draw a rectangle on the current location of the mouse pointer.
        cv2.rectangle(img=canvas, pt1=(x-50,y-50), pt2=(x+50,y+50), 
                      color=(113,182,255), thickness=-1)

    # Check if the middle mouse button is pressed.
    elif event == cv2.EVENT_MBUTTONDOWN:
        
        # Clear the canvas.
        canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                                 int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                          dtype=np.uint8)

In [4]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
# camera_video.set(3,1280)
# camera_video.set(4,960)
 
# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)
 
# Create a named resizable window.
# This line is added to re-create the window,
# in case you have closed the window created in the cell above.
cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)
 
# Attach the mouse callback function to the window.
cv2.setMouseCallback('Webcam Feed', drawShapes)
 
# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Update the pixel values of the frame with the canvas's values at the indexes where canvas!=0
    # i.e. where canvas is not black and something is drawn there.
    # In short, this will copy the shapes from canvas to the frame.
    frame[np.mean(canvas, axis=2)!=0] = canvas[np.mean(canvas, axis=2)!=0]
    
    # Display the frame.
    cv2.imshow('Webcam Feed', frame)   
 
    # Check if 'ESC' is pressed and break the loop.
    if cv2.waitKey(20) & 0xFF == ord('q'): # TODO: what does & do in python? check the video 
        break
        
# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()

In [5]:
def drawResizableShapes(event, x, y, flags, userdata):
    '''
    This function will draw circle and rectangle on a canvas and clear it
    on different mouse events.
    Args:
        event:    The mouse event that is captured.
        x:        The x-coordinate of the mouse pointer position on the window.
        y:        The y-coordinate of the mouse pointer position on the window.
        flags:    It is one of the MouseEventFlags constants.
        userdata: The parameter passed from the `cv2.setMouseCallback()` function.
    '''
    
    # Access the needed variables from outside of the current scope.
    global start_x, start_y, canvas, draw_shape
    
    # Check if the left mouse button is pressed.
    if event == cv2.EVENT_LBUTTONDOWN:
        
        # Enable the draw circle mode.
        draw_shape = 'Circle'
        
        # Set the start x and y to the current x and y values.
        start_x = x
        start_y = y
        
    # Check if the left mouse button is pressed.
    elif event == cv2.EVENT_RBUTTONDOWN:
        
        # Enable the draw rectangle mode.
        draw_shape = 'Rectangle'
        
        # Set the start x and y to the current x and y values.
        start_x = x
        start_y = y
         
    # Check if the mouse has moved on the window.
    elif event == cv2.EVENT_MOUSEMOVE:
        
        # Get the pointer x-coordinate distance between start and current point.
        pointer_pos_diff_x = abs(start_x-x)
        
        # Get the pointer y-coordinate distance between start and current point.
        pointer_pos_diff_y = abs(start_y-y)
        
        # Check if the draw circle mode is enabled.
        if draw_shape == 'Circle':
            
            # Draw a circle on the start x and y coordinates,
            # of size depending upon the distance between start,
            # and current x and y coordinates.
            cv2.circle(img = canvas, center = (start_x, start_y), 
                       radius = pointer_pos_diff_x + pointer_pos_diff_y,
                       color = (113,182,255), thickness = -1)
            
        # Check if the draw rectangle mode is enabled.
        elif draw_shape == 'Rectangle':
            
            # Draw a rectangle on the start x and y coordinates,
            # of size depending upon the distance between start,
            # and current x and y coordinates.
            cv2.rectangle(img=canvas, pt1=(start_x-pointer_pos_diff_x,
                                           start_y-pointer_pos_diff_y),
                          pt2=(start_x+pointer_pos_diff_x, start_y+pointer_pos_diff_y), 
                          color=(113,182,255), thickness=-1)
            
    # Check if the left or right mouse button is released.
    elif event == cv2.EVENT_LBUTTONUP or event == cv2.EVENT_RBUTTONUP:
        
        # Disable the draw shapes mode.
        draw_shape = None
        
    # Check if the middle mouse button is pressed.
    elif event == cv2.EVENT_MBUTTONDOWN:
        
        # Clear the canvas.
        canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                                 int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                          dtype=np.uint8)

In [ ]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)

# Create a named resizable window.
cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

# Attach the mouse callback function to the window.
cv2.setMouseCallback('Webcam Feed', drawResizableShapes)

# Initialize variables to store start mouse pointer x and y location.
start_x = 0
start_y = 0

# Initialize a variable to store the draw shape mode.
draw_shape = None

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Update the pixel values of the frame with the canvas's values at the indexes where canvas!=0
    # i.e. where canvas is not black and something is drawn there.
    # In short, this will copy the shapes from canvas to the frame.
    frame[np.mean(canvas, axis=2)!=0] = canvas[np.mean(canvas, axis=2)!=0]
    
    # Display the frame.
    cv2.imshow('Webcam Feed', frame)   

    # Check if 'ESC' is pressed and break the loop.
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
        
# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()

In [ ]:
def draw(event, x, y, flags, userdata):
    '''
    This function will select paint color, draw and clear a canvas 
    based on different mouse events.
    Args:
        event:    The mouse event that is captured.
        x:        The x-coordinate of the mouse pointer position on the window.
        y:        The y-coordinate of the mouse pointer position on the window.
        flags:    It is one of the MouseEventFlags constants.
        userdata: The parameter passed from the `cv2.setMouseCallback()` function.
    '''
    
    # Access the needed variables from outside of the current scope.
    global prev_x, prev_y, canvas, mode, color
    
    # Check if the left mouse button is double-clicked.
    if event == cv2.EVENT_LBUTTONDBLCLK:
        
        # Check if the mouse pointer y-coordinate is less than equal to a certain threshold.
        if y <= 10 + rect_height:
            
            # Check if the mouse pointer x-coordinate is over the orange color rectangle.
            if x&gt;(frame_width//1.665-rect_width//2) and \
            x&lt;(frame_width//1.665-rect_width//2)+rect_width: 
                
                # Update the color variable value to orange.
                color = 113, 182, 255
            
            # Check if the mouse pointer x-coordinate is over the pink color rectangle.
            elif x&gt;(int(frame_width//2)-rect_width//2) and \
            x&lt;(int(frame_width//2)-rect_width//2)+rect_width:
                
                # Update the color variable value to pink.
                color = 203, 192, 255
            
            # Check if the mouse pointer x-coordinate is over the yellow color rectangle.
            elif x&gt;(int(frame_width//2.5)-rect_width//2) and \
            x&lt;(int(frame_width//2.5)-rect_width//2)+rect_width:
                
                # Update the color variable value to yellow.
                color = 0, 255, 255
    
    # Check if the left mouse button is pressed.
    elif event == cv2.EVENT_LBUTTONDOWN:
        
        # Enable the paint mode.
        mode = 'Paint'
        
    # Check if the right mouse button is pressed.
    elif event == cv2.EVENT_RBUTTONDOWN:
        
        # Enable the paint mode.
        mode = 'Erase'
        
    # Check if the left or right mouse button is released.
    elif event == cv2.EVENT_LBUTTONUP or event == cv2.EVENT_RBUTTONUP:
        
        # Disable the active mode.
        mode = None
        
        # Reset by updating the previous x and y values to None.
        prev_x = None
        prev_y = None        
    
    # Check if the mouse has moved on the window.
    elif event == cv2.EVENT_MOUSEMOVE:
        
        # Check if a mode is enabled and the previous x and y donot have valid values.
        if mode and (not (prev_x and prev_y)):
            # Set the previous x and y to the current x and y values.
            prev_x = x
            prev_y = y
        # Check if the paint mode is enabled.
        if mode == 'Paint':
            
            # Draw a line from previous x and y to the current x and y.
            cv2.line(img=canvas, pt1=(x,y), pt2=(prev_x,prev_y), color=color, thickness=10)
        
        # Check if the erase mode is enabled.
        elif mode == 'Erase':
        
            # Draw a black line from previous x and y to the current x and y.
            # This will erase the paint between previous x and y and the current x and y.
            cv2.line(img=canvas, pt1=(x,y), pt2=(prev_x,prev_y), color=(0,0,0), thickness=20)
            
        # Update the previous x and y to the current x and y values.
        prev_x = x
        prev_y = y
        
    # Check if the middle mouse button is pressed.
    elif event == cv2.EVENT_MBUTTONDOWN:
        
        # Clear the canvas.
        canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                                 int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                          dtype=np.uint8)

In [ ]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)

# Create a named resizable window.
cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)

# Attach the mouse callback function to the window.
cv2.setMouseCallback('Webcam Feed', draw)

# Initialize variables to store previous mouse pointer x and y location.
prev_x = None
prev_y = None

# Initialize a variable to store the active mode.
mode = None

# Initialize a variable to store the color value.
color = 203, 192, 255

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
        
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Get the colors rectangles previews height and width.
    rect_height, rect_width = int(frame_height/10), int(frame_width/10)
    
    # Update the pixel values of the frame with the canvas's values at the indexes where canvas!=0
    # i.e. where canvas is not black and something is drawn there.
    # In short, this will copy the drawings from canvas to the frame.
    frame[np.mean(canvas, axis=2)!=0] = canvas[np.mean(canvas, axis=2)!=0]
    
    # Overlay the colors previews rectangles over the frame.
    ###################################################################################################################
    
    # Overlay the orange color preview on the frame.
    cv2.rectangle(img=frame, pt1=(int((frame_width//1.665)-rect_width//2), 10),
                  pt2=(int((frame_width//1.665)+rect_width//2), 10+rect_height),
                  color=(113, 182, 255), thickness=-1)
    
    # Draw an outline around the orange color preview.
    cv2.rectangle(img=frame, pt1=(int((frame_width//1.665)-rect_width//2), 10),
                  pt2=(int((frame_width//1.665)+rect_width//2), 10+rect_height),
                  color=(255, 255, 255), thickness=2)
    
    # Overlay the pink color preview on the frame.
    cv2.rectangle(img=frame, pt1=(int((frame_width//2)-rect_width//2), 10),
                  pt2=(int((frame_width//2)+rect_width//2), 10+rect_height),
                  color=(203, 192, 255), thickness=-1)
    
    # Draw an outline around the pink color preview.
    cv2.rectangle(img=frame, pt1=(int((frame_width//2)-rect_width//2), 10),
                  pt2=(int((frame_width//2)+rect_width//2), 10+rect_height),
                  color=(255, 255, 255), thickness=2)
    
    # Overlay the yellow color preview on the frame.
    cv2.rectangle(img=frame, pt1=(int((frame_width//2.5)-rect_width//2), 10),
                  pt2=(int((frame_width//2.5)+rect_width//2), 10+rect_height),
                  color=(0, 255, 255), thickness=-1)
    
    # Draw an outline around the yellow color preview.
    cv2.rectangle(img=frame, pt1=(int((frame_width//2.5)-rect_width//2), 10),
              pt2=(int((frame_width//2.5)+rect_width//2), 10+rect_height),
              color=(255, 255, 255), thickness=2)
    
    ###################################################################################################################
    
    # Display the frame.
    cv2.imshow('Webcam Feed', frame)   

    # Check if 'ESC' is pressed and break the loop.
    if cv2.waitKey(20) &amp;  0xFF == 27:
        break
        
# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()

### Creating Warm Filter-like Effect
* Absorbs blue cast in images, and improves skin tones.
* This is achieved by decreasing the pixel intensities of the blue channel and increase the intensities of the red channel of an image.
* We will use the 'Look Up Tables'